In [1]:
import numpy as np
import pandas as pd
import os

Set current working directory to the right folder and load in data

In [2]:
from google.colab import drive

# drive.mount('/content/drive/')
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [3]:
os.chdir('./drive/MyDrive/ML DS539/CoLab/Data Cleaning')

In [4]:
df = pd.read_csv("database.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Part 1: Aggergating the data

Set up lists for data cleaning later

In [5]:
all_years = df["Year"].unique()
states = df["State"].unique()

In [6]:
months = df["Month"].unique()
crime_types = df["Crime Type"].unique()
record_sources = df["Record Source"].unique()
agency_types = df["Agency Type"].unique()

In [7]:
vic_races = df["Victim Race"].unique()
vic_ethic = df["Victim Ethnicity"].unique()
perp_races = df["Perpetrator Race"].unique()
perp_ethic = df["Perpetrator Ethnicity"].unique()
relationships = df["Relationship"].unique()
weapons = df["Weapon"].unique()

In [8]:
relationships = {'friend':['Acquaintance','Friend','Neighbor','Employer','Employee'],
'stranger':['Unknown','Stranger'],
'spouse':['Wife','Ex-Husband','Husband','Ex-Wife','Common-Law Husband','Common-Law Wife'],
'dating':['Girlfriend','Boyfriend','Boyfriend/Girlfriend'],
'sibling':['Brother','Sister'],
'child':['Stepdaughter','Son','Daughter','Stepson'],
'parent':['Father','Mother','Stepfather','Stepmother'],
'other_family':['Family','In-Law']}

Clean out empty cells

In [9]:
len(df[df.isnull().any(1)]) #no NaN values so we are good!
df.loc[np.where(df["Perpetrator Age"] == " ")[0], "Perpetrator Age"] = "0"

Aggergate Crime counts

In [20]:
def create_new_row(state,state_df, year, agg_df):
  new_row = [np.nan for i in range(0,178)]

  temp_df = state_df[state_df["Year"] == year]
  new_row[0]= state
  new_row[1] = year

  # col_names.append("state")
  # col_names.append("year")
  no_of_crimes = len(temp_df)
  if no_of_crimes ==0:
    new_row = [state, year] 
    new_row = new_row + [0 for i in range(0, 196)]
    agg_df = agg_df.append(pd.Series(new_row), ignore_index=True)
    return agg_df
    
  new_row[2]= no_of_crimes 
  # col_names.append("no_of_crimes")
  no_of_cities  = len(temp_df["City"].unique())
  new_row[3]= no_of_cities
  # col_names.append("no_of_cities")

  #Solved Status
  no_solved = len(temp_df[temp_df["Crime Solved"] == "Yes"])
  new_row[4] = no_solved
  # col_names.append("no_of_crime_solved")
  new_row[5] = no_solved/no_of_crimes
  # col_names.append("rate_of_crime_solved")

  # col_names.append("no_of_crime_unsolved")
  no_unsolved = len(temp_df[temp_df["Crime Solved"] == "No"])
  new_row[6] = no_unsolved
  new_row[7] = no_unsolved/no_of_crimes
  # col_names.append("rate_of_crime_unsolved")


  #Age Values
  new_row[8] = np.mean([int(i) for i in temp_df["Perpetrator Age"]])
  # col_names.append("avg_perp_age")
  new_row[9] = min([int(i) for i in temp_df["Perpetrator Age"]])
  # col_names.append("min_avg_age")
  new_row[10] = max([int(i) for i in temp_df["Perpetrator Age"]])
  # col_names.append("max_perp_age")

  new_row[11] = np.mean(temp_df["Victim Age"])
  # col_names.append("mean_vic_age")
  new_row[12] = min(temp_df["Victim Age"])
  # col_names.append("min_vic_age")
  new_row[13] = max(temp_df["Victim Age"])
  # col_names.append("max_vic_age")


  #GENDERS
  new_row[14] = len(temp_df[temp_df["Perpetrator Sex"] == "Male"])
  # col_names.append("perp_male_count")
  new_row[15] = new_row[14]/no_of_crimes
  # col_names.append("perp_male_rate")

  new_row[16] = len(temp_df[temp_df["Perpetrator Sex"] == "Female"])
  # col_names.append("perp_female_count")/
  new_row[17] = new_row[16]/no_of_crimes
  # col_names.append("perp_female_rate")

  new_row[18] = len(temp_df[temp_df["Perpetrator Sex"] == "Unknown"])
  # col_names.append("perp_unknown_count")
  new_row[19]  = new_row[18] /no_of_crimes
  # col_names.append("perp_unknow_rate")

  new_row[20] = len(temp_df[temp_df["Victim Sex"] == "Male"])
  # col_names.append("vic_male_count")
  new_row[21] = new_row[20]/no_of_crimes
  # col_names.append("vic_male_count")

  new_row[22] = len(temp_df[temp_df["Victim Sex"] == "Female"])
  # col_names.append("vic_female_count")
  new_row[23] = new_row[22]/no_of_crimes
  # col_names.append("vic_female_rate")

  new_row[24] = len(temp_df[temp_df["Victim Sex"] == "Unknown"])
  # col_names.append("vic_unknown_count")
  new_row[25] = new_row[24]/no_of_crimes
  # col_names.append("vic_unknown_rate")

  #Perp/Vic Count

  new_row[26] = np.mean(temp_df["Perpetrator Count"])
  # col_names.append("mean_perp_count")
  new_row[27] = min(temp_df["Perpetrator Count"])
  # col_names.append("min_perp_count")
  new_row[28] = max(temp_df["Perpetrator Count"])
  # col_names.append("max_perp_count")

  new_row[29] = np.mean(temp_df["Victim Count"])
  # col_names.append("mean_vic_count")
  new_row[30] = min(temp_df["Victim Count"])
  # col_names.append("min_vic_count")
  new_row[31] = max(temp_df["Victim Count"])
  # col_names.append("max_vic_count")

  j = 32
  for month in months:
    month_count = len(temp_df[temp_df["Month"] ==  month])
    new_row[j] = month_count
    # col_names.append(month+"_count")
    j+=1
    new_row[j] = month_count/no_of_crimes
    # col_names.append(month+"_rate")
    j+=1

  for c_type in crime_types:
    type_count = len(temp_df[temp_df["Crime Type"] == c_type])
    # col_names.append(c_type+"_count")
    new_row[j] = type_count
    j+=1
    new_row[j] = type_count/no_of_crimes
    # col_names.append(c_type+"_rate")
    j+=1


  for r in record_sources:
    rec_count = len(temp_df[temp_df["Record Source"] == r])
    # col_names.append(r+"_count")
    new_row[j] = rec_count
    j+=1
    new_row[j] =  rec_count/no_of_crimes
    # col_names.append(r+"_rate")
    j+=1

  for r in agency_types :
    agency_t = len(temp_df[temp_df["Agency Type"] == r])
    # col_names.append(r+"_count")
    new_row[j] = agency_t
    j+=1
    new_row[j] =  agency_t/no_of_crimes
    # col_names.append(r+"_rate")
    j+=1

  for r in vic_races:
    vic_r = len(temp_df[temp_df["Victim Race"] == r])
    # col_names.append(r+"_vic_count")
    new_row[j] = vic_r
    j+=1
    new_row[j] =  vic_r/no_of_crimes
    # col_names.append(r+"_vic_rate")
    j+=1

  for r in vic_ethic:
    vic_e = len(temp_df[temp_df["Victim Ethnicity"] == r])
    # col_names.append(r+"_vic_count")
    new_row[j] = vic_e
    j+=1
    new_row[j] =  vic_e/no_of_crimes
    # col_names.append(r+"_vic_rate")
    j+=1

  for r in perp_races:
    perp_r = len(temp_df[temp_df["Perpetrator Race"] == r])
    # col_names.append(r+"_perp_count")
    new_row[j] = perp_r
    j+=1
    new_row[j] =  perp_r/no_of_crimes
    # col_names.append(r+"_perp_rate")
    j+=1

  for r in perp_ethic:
    perp_e = len(temp_df[temp_df["Perpetrator Ethnicity"] == r])
    # col_names.append(r+"_perp_count")
    new_row[j] = perp_e
    j+=1
    new_row[j] =  perp_e/no_of_crimes
    # col_names.append(r+"_perp_rate")
    j+=1

  for r in relationships.keys():
    rel = len(temp_df[temp_df["Relationship"].isin(relationships[r])])
    # col_names.append(r+"_count")
    new_row[j] = rel
    j+=1
    new_row[j] =  rel/no_of_crimes
    # col_names.append(r+"_rate")
    j+=1

  for r in weapons:
    wep = len(temp_df[temp_df["Weapon"] == r])
    # col_names.append(r+"_count")
    new_row[j] = wep
    j+=1
    new_row[j] =  wep/no_of_crimes
    # col_names.append(r+"_rate")
    j+=1
  
  agg_df = agg_df.append(pd.Series(new_row), ignore_index=True)
  return agg_df

In [21]:
agg_df = pd.DataFrame()
# col_names = []

In [22]:
for state in states:
  state_df = df[df["State"] == state]
  print(state)
  for year in all_years:
    agg_df = create_new_row(state,state_df, year, agg_df)

Alaska
Alabama
Arkansas
Arizona
California
Colorado
Connecticut
District of Columbia
Delaware
Florida
Georgia
Hawaii
Iowa
Idaho
Illinois
Indiana
Kansas
Kentucky
Louisiana
Massachusetts
Maryland
Maine
Michigan
Minnesota
Missouri
Mississippi
Montana
Nebraska
North Carolina
North Dakota
New Hampshire
New Jersey
New Mexico
Nevada
New York
Ohio
Oklahoma
Oregon
Pennsylvania
Rhodes Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Virginia
Vermont
Washington
Wisconsin
West Virginia
Wyoming


In [33]:
agg_df = agg_df.iloc[:, :158]

In [34]:
agg_df.columns = ['state',
 'year',
 'no_of_crimes',
 'no_of_cities',
 'no_of_crime_solved',
 'rate_of_crime_solved',
 'no_of_crime_unsolved',
 'rate_of_crime_unsolved',
 'avg_perp_age',
 'min_avg_age',
 'max_perp_age',
 'mean_vic_age',
 'min_vic_age',
 'max_vic_age',
 'perp_male_count',
 'perp_male_rate',
 'perp_female_count',
 'perp_female_rate',
 'perp_unknown_gender_count',
 'perp_unknow_gender_rate',
 'vic_male_count',
 'vic_male_count',
 'vic_female_count',
 'vic_female_rate',
 'vic_unknown_gender_count',
 'vic_unknown_gender_rate',
 'mean_perp_count',
 'min_perp_count',
 'max_perp_count',
 'mean_vic_count',
 'min_vic_count',
 'max_vic_count',
 'January_count',
 'January_rate',
 'March_count',
 'March_rate',
 'April_count',
 'April_rate',
 'May_count',
 'May_rate',
 'June_count',
 'June_rate',
 'July_count',
 'July_rate',
 'August_count',
 'August_rate',
 'December_count',
 'December_rate',
 'November_count',
 'November_rate',
 'February_count',
 'February_rate',
 'October_count',
 'October_rate',
 'September_count',
 'September_rate',
 'Murder or Manslaughter_count',
 'Murder or Manslaughter_rate',
 'Manslaughter by Negligence_count',
 'Manslaughter by Negligence_rate',
 'FBI_count',
 'FBI_rate',
 'FOIA_count',
 'FOIA_rate',
 'Municipal Police_count',
 'Municipal Police_rate',
 'County Police_count',
 'County Police_rate',
 'State Police_count',
 'State Police_rate',
 'Sheriff_count',
 'Sheriff_rate',
 'Special Police_count',
 'Special Police_rate',
 'Regional Police_count',
 'Regional Police_rate',
 'Tribal Police_count',
 'Tribal Police_rate',
 'Native American/Alaska Native_vic_count',
 'Native American/Alaska Native_vic_rate',
 'White_vic_count',
 'White_vic_rate',
 'Black_vic_count',
 'Black_vic_rate',
 'Unknown_race_vic_count',
 'Unknown_race_vic_rate',
 'Asian/Pacific Islander_vic_count',
 'Asian/Pacific Islander_vic_rate',
 'Unknown_ethic_vic_count',
 'Unknown_ethic_vic_rate',
 'Not Hispanic_vic_count',
 'Not Hispanic_vic_rate',
 'Hispanic_vic_count',
 'Hispanic_vic_rate',
 'Native American/Alaska Native_perp_count',
 'Native American/Alaska Native_perp_rate',
 'White_perp_count',
 'White_perp_rate',
 'Unknown_race_perp_count',
 'Unknown_race_perp_rate',
 'Black_perp_count',
 'Black_perp_rate',
 'Asian/Pacific Islander_perp_count',
 'Asian/Pacific Islander_perp_rate',
 'Unknown_ethic_perp_count',
 'Unknown_ethic_perp_rate',
 'Not Hispanic_perp_count',
 'Not Hispanic_perp_rate',
 'Hispanic_perp_count',
 'Hispanic_perp_rate',
 'friend_count',
 'friend_rate',
 'stranger_count',
 'stranger_rate',
 'spouse_count',
 'spouse_rate',
 'dating_count',
 'dating_rate',
 'sibling_count',
 'sibling_rate',
'child_count',
'child_rate',
'parent_count',
'parent_rate',
'other_family_count',
'other_family_rate',
 'Blunt Object_count',
 'Blunt Object_rate',
 'Strangulation_count',
 'Strangulation_rate',
 'Unknown_count',
 'Unknown_rate',
 'Rifle_count',
 'Rifle_rate',
 'Knife_count',
 'Knife_rate',
 'Firearm_count',
 'Firearm_rate',
 'Shotgun_count',
 'Shotgun_rate',
 'Fall_count',
 'Fall_rate',
 'Handgun_count',
 'Handgun_rate',
 'Drowning_count',
 'Drowning_rate',
 'Suffocation_count',
 'Suffocation_rate',
 'Explosives_count',
 'Explosives_rate',
 'Fire_count',
 'Fire_rate',
 'Drugs_count',
 'Drugs_rate',
 'Gun_count',
 'Gun_rate',
 'Poison_count',
 'Poison_rate']

In [35]:
agg_df.to_csv("Agg_df.csv")

#### Part 2: Calculating Crime rate

In [36]:
agg_df = pd.read_csv("Agg_df.csv", index_col= 'Unnamed: 0')

In [37]:
state_pop = pd.read_csv("state_pops.csv", index_col= "Year")

In [38]:
agg_df["Crime Rate"] = 0 

In [39]:
for i in range(0, len(agg_df)):
  pop = state_pop.loc[agg_df.loc[i, "year"], agg_df.loc[i, "state"]]*1000000
  agg_df.loc[i, "Crime Rate"] = (agg_df.loc[i, "no_of_crimes"]/pop)*100000

In [40]:
agg_df

,state,year,no_of_crimes,no_of_cities,no_of_crime_solved,rate_of_crime_solved,no_of_crime_unsolved,rate_of_crime_unsolved,avg_perp_age,min_avg_age,max_perp_age,mean_vic_age,min_vic_age,max_vic_age,perp_male_count,perp_male_rate,perp_female_count,perp_female_rate,perp_unknown_gender_count,perp_unknow_gender_rate,vic_male_count,vic_male_count.1,vic_female_count,vic_female_rate,vic_unknown_gender_count,vic_unknown_gender_rate,mean_perp_count,min_perp_count,max_perp_count,mean_vic_count,min_vic_count,max_vic_count,January_count,January_rate,March_count,March_rate,April_count,April_rate,May_count,May_rate,...,sibling_rate,child_count,child_rate,parent_count,parent_rate,other_family_count,other_family_rate,Blunt Object_count,Blunt Object_rate,Strangulation_count,Strangulation_rate,Unknown_count,Unknown_rate,Rifle_count,Rifle_rate,Knife_count,Knife_rate,Firearm_count,Firearm_rate,Shotgun_count,Shotgun_rate,Fall_count,Fall_rate,Handgun_count,Handgun_rate,Drowning_count,Drowning_rate,Suffocation_count,Suffocation_rate,Explosives_count,Explosives_rate,Fire_count,Fire_rate,Drugs_count,Drugs_rate,Gun_count,Gun_rate,Poison_count,Poison_rate,Crime Rate
0,Alaska,1980.0,47.0,6.0,37.0,0.787234,10.0,0.212766,25.787234,0.0,61.0,31.255319,8.0,99.0,33.0,0.702128,4.0,0.085106,10.0,0.212766,33.0,0.702128,14.0,0.297872,0.0,0.0,0.085106,0.0,1.0,0.127660,0.0,2.0,1.0,0.021277,2.0,0.042553,3.0,0.063830,3.0,0.063830,...,0.106383,3.0,0.063830,0.0,0.000000,1.0,0.021277,4.0,0.085106,2.0,0.042553,5.0,0.106383,15.0,0.319149,5.0,0.106383,4.0,0.085106,2.0,0.042553,1.0,0.021277,8.0,0.170213,1.0,0.021277,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.0,11.595919
1,Alaska,1981.0,69.0,8.0,62.0,0.898551,7.0,0.101449,26.028986,0.0,57.0,29.942029,0.0,71.0,55.0,0.797101,7.0,0.101449,7.0,0.101449,44.0,0.637681,25.0,0.362319,0.0,0.0,0.115942,0.0,2.0,0.260870,0.0,2.0,6.0,0.086957,7.0,0.101449,1.0,0.014493,14.0,0.202899,...,0.101449,6.0,0.086957,4.0,0.057971,1.0,0.014493,1.0,0.014493,2.0,0.028986,2.0,0.028986,15.0,0.217391,7.0,0.101449,1.0,0.014493,10.0,0.144928,0.0,0.000000,27.0,0.391304,2.0,0.028986,1.0,0.014493,0.0,0.0,1.0,0.014493,0.0,0.000000,0.0,0.00,0.0,0.0,16.487810
2,Alaska,1982.0,75.0,10.0,47.0,0.626667,28.0,0.373333,17.640000,0.0,61.0,28.800000,0.0,85.0,42.0,0.560000,5.0,0.066667,28.0,0.373333,52.0,0.693333,23.0,0.306667,0.0,0.0,0.200000,0.0,2.0,1.013333,0.0,7.0,3.0,0.040000,5.0,0.066667,9.0,0.120000,9.0,0.120000,...,0.053333,2.0,0.026667,1.0,0.013333,3.0,0.040000,7.0,0.093333,3.0,0.040000,4.0,0.053333,11.0,0.146667,9.0,0.120000,2.0,0.026667,2.0,0.026667,0.0,0.000000,36.0,0.480000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.0,0.013333,0.0,0.000000,0.0,0.00,0.0,0.0,16.681272
3,Alaska,1983.0,74.0,8.0,63.0,0.851351,11.0,0.148649,26.216216,0.0,57.0,32.202703,0.0,72.0,60.0,0.810811,3.0,0.040541,11.0,0.148649,49.0,0.662162,25.0,0.337838,0.0,0.0,0.081081,0.0,2.0,0.567568,0.0,5.0,5.0,0.067568,15.0,0.202703,6.0,0.081081,5.0,0.067568,...,0.040541,1.0,0.013514,4.0,0.054054,4.0,0.054054,6.0,0.081081,2.0,0.027027,5.0,0.067568,16.0,0.216216,11.0,0.148649,1.0,0.013514,11.0,0.148649,0.0,0.000000,22.0,0.297297,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.0,15.150988
4,Alaska,1984.0,52.0,5.0,45.0,0.865385,7.0,0.134615,23.288462,0.0,47.0,34.634615,2.0,99.0,36.0,0.692308,9.0,0.173077,7.0,0.134615,41.0,0.788462,11.0,0.211538,0.0,0.0,0.134615,0.0,2.0,0.884615,0.0,6.0,2.0,0.038462,5.0,0.096154,3.0,0.057692,12.0,0.230769,...,0.057692,1.0,0.019231,3.0,0.057692,0.0,0.000000,5.0,0.096154,2.0,0.038462,1.0,0.019231,11.0,0.211538,7.0,0.134615,0.0,0.000000,2.0,0.038462,0.0,0.000000,22.0,0.423077,0.0,0.000000,0.0,0.000000,0.0,0.0,2.0,0.038462,0.0,0.000000,0.0,0.00,0.0,0.0,10.122600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

#### Part 3: Setting up y-variables

In [41]:
states = agg_df["state"].unique()
years = agg_df["year"].unique()

In [42]:
agg_df["Top_10"] = 0
agg_df["Next_Year_Top_10"] = np.nan
agg_df["Next_Year_Crime_Rate"] = np.nan

In [43]:
years

array([1980., 1981., 1982., 1983., 1984., 1985., 1986., 1987., 1988.,
       1989., 1990., 1991., 1992., 1993., 1994., 1995., 1996., 1997.,
       1998., 1999., 2000., 2001., 2002., 2003., 2004., 2005., 2006.,
       2007., 2008., 2009., 2010., 2011., 2012., 2013., 2014.])

In [44]:
for year in years:
  top_10_index = agg_df.loc[np.where(agg_df["year"]==year)[0], :].nlargest(10, "Crime Rate").index
  agg_df.loc[top_10_index, "Top_10"] = 1

In [45]:
agg_df["Top_10"].value_counts()

0    1435
1     350
Name: Top_10, dtype: int64

In [46]:
for state in states:
  indicies = agg_df[agg_df["state"] == state].index
  for i in range(0, len(indicies)-1):
    agg_df.loc[indicies[i], "Next_Year_Crime_Rate"]  = agg_df.loc[indicies[i+1], "Crime Rate"]
    agg_df.loc[indicies[i], "Next_Year_Top_10"]  = agg_df.loc[indicies[i+1], "Top_10"]

In [47]:
from sklearn.preprocessing import OneHotEncoder

In [48]:
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(agg_df[['state']]).toarray())
enc_df.columns = [j+"_one_hot" for j in agg_df["state"].unique()]

In [49]:
agg_df

,state,year,no_of_crimes,no_of_cities,no_of_crime_solved,rate_of_crime_solved,no_of_crime_unsolved,rate_of_crime_unsolved,avg_perp_age,min_avg_age,max_perp_age,mean_vic_age,min_vic_age,max_vic_age,perp_male_count,perp_male_rate,perp_female_count,perp_female_rate,perp_unknown_gender_count,perp_unknow_gender_rate,vic_male_count,vic_male_count.1,vic_female_count,vic_female_rate,vic_unknown_gender_count,vic_unknown_gender_rate,mean_perp_count,min_perp_count,max_perp_count,mean_vic_count,min_vic_count,max_vic_count,January_count,January_rate,March_count,March_rate,April_count,April_rate,May_count,May_rate,...,parent_count,parent_rate,other_family_count,other_family_rate,Blunt Object_count,Blunt Object_rate,Strangulation_count,Strangulation_rate,Unknown_count,Unknown_rate,Rifle_count,Rifle_rate,Knife_count,Knife_rate,Firearm_count,Firearm_rate,Shotgun_count,Shotgun_rate,Fall_count,Fall_rate,Handgun_count,Handgun_rate,Drowning_count,Drowning_rate,Suffocation_count,Suffocation_rate,Explosives_count,Explosives_rate,Fire_count,Fire_rate,Drugs_count,Drugs_rate,Gun_count,Gun_rate,Poison_count,Poison_rate,Crime Rate,Top_10,Next_Year_Top_10,Next_Year_Crime_Rate
0,Alaska,1980.0,47.0,6.0,37.0,0.787234,10.0,0.212766,25.787234,0.0,61.0,31.255319,8.0,99.0,33.0,0.702128,4.0,0.085106,10.0,0.212766,33.0,0.702128,14.0,0.297872,0.0,0.0,0.085106,0.0,1.0,0.127660,0.0,2.0,1.0,0.021277,2.0,0.042553,3.0,0.063830,3.0,0.063830,...,0.0,0.000000,1.0,0.021277,4.0,0.085106,2.0,0.042553,5.0,0.106383,15.0,0.319149,5.0,0.106383,4.0,0.085106,2.0,0.042553,1.0,0.021277,8.0,0.170213,1.0,0.021277,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.0,11.595919,1,1.0,16.487810
1,Alaska,1981.0,69.0,8.0,62.0,0.898551,7.0,0.101449,26.028986,0.0,57.0,29.942029,0.0,71.0,55.0,0.797101,7.0,0.101449,7.0,0.101449,44.0,0.637681,25.0,0.362319,0.0,0.0,0.115942,0.0,2.0,0.260870,0.0,2.0,6.0,0.086957,7.0,0.101449,1.0,0.014493,14.0,0.202899,...,4.0,0.057971,1.0,0.014493,1.0,0.014493,2.0,0.028986,2.0,0.028986,15.0,0.217391,7.0,0.101449,1.0,0.014493,10.0,0.144928,0.0,0.000000,27.0,0.391304,2.0,0.028986,1.0,0.014493,0.0,0.0,1.0,0.014493,0.0,0.000000,0.0,0.00,0.0,0.0,16.487810,1,1.0,16.681272
2,Alaska,1982.0,75.0,10.0,47.0,0.626667,28.0,0.373333,17.640000,0.0,61.0,28.800000,0.0,85.0,42.0,0.560000,5.0,0.066667,28.0,0.373333,52.0,0.693333,23.0,0.306667,0.0,0.0,0.200000,0.0,2.0,1.013333,0.0,7.0,3.0,0.040000,5.0,0.066667,9.0,0.120000,9.0,0.120000,...,1.0,0.013333,3.0,0.040000,7.0,0.093333,3.0,0.040000,4.0,0.053333,11.0,0.146667,9.0,0.120000,2.0,0.026667,2.0,0.026667,0.0,0.000000,36.0,0.480000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.0,0.013333,0.0,0.000000,0.0,0.00,0.0,0.0,16.681272,1,1.0,15.150988
3,Alaska,1983.0,74.0,8.0,63.0,0.851351,11.0,0.148649,26.216216,0.0,57.0,32.202703,0.0,72.0,60.0,0.810811,3.0,0.040541,11.0,0.148649,49.0,0.662162,25.0,0.337838,0.0,0.0,0.081081,0.0,2.0,0.567568,0.0,5.0,5.0,0.067568,15.0,0.202703,6.0,0.081081,5.0,0.067568,...,4.0,0.054054,4.0,0.054054,6.0,0.081081,2.0,0.027027,5.0,0.067568,16.0,0.216216,11.0,0.148649,1.0,0.013514,11.0,0.148649,0.0,0.000000,22.0,0.297297,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.0,0.0,15.150988,1,1.0,10.122600
4,Alaska,1984.0,52.0,5.0,45.0,0.865385,7.0,0.134615,23.288462,0.0,47.0,34.634615,2.0,99.0,36.0,0.692308,9.0,0.173077,7.0,0.134615,41.0,0.788462,11.0,0.211538,0.0,0.0,0.134615,0.0,2.0,0.884615,0.0,6.0,2.0,0.038462,5.0,0.096154,3.0,0.057692,12.0,0.230769,...,3.0,0.057692,0.0,0.000000,5.0,0.096154,2.0,0.038462,1.0,0.019231,11.0,0.211538,7.0,0.134615,0.0,0.000000,2.0,0.038462,0.0,0.000000,22.0,0.423077,0.0,0.000000,0.0,0.000000,0.0,0.0,2.0,0.038462,0.0,0.000000,0.0,0.00,0.0,0.0,10.122600,1,1.0,9.389760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [50]:
# generate binary values using get_dummies
dum_df = pd.get_dummies(agg_df['state'])

In [51]:
final_df =  dum_df.join(agg_df)

In [52]:
final_df.to_csv("pre_split_df.csv")

#### Part 4: Splitting into testing and training

In [53]:
len(years) -1 #can't reliably use 2014

34

In [54]:
training_years = [i for i in range(1980, 2009)]
print(len(training_years))
testing_years = [i for i in range(2009, 2014)]
print(len(testing_years))

29
5


In [55]:
testing_years

[2009, 2010, 2011, 2012, 2013]

In [56]:
testing_df = final_df[final_df["year"].isin(testing_years)]

# df[~df['A'].isin([3, 6])]

In [57]:
len(testing_df)

255

In [58]:
51*5

255

In [59]:
training_df = final_df[final_df["year"].isin(training_years)]
len(training_df)

1479

In [60]:
51*29

1479

In [65]:
# training_df.to_csv("training.csv")
# testing_df.to_csv("testing.csv")